In [9]:
import os
import json
import re
from typing import List, Dict, Any, Optional

os.environ["DEEPSEEK_API_KEY"] = "" 

In [10]:
# ============== 一、LLM 客户端封装 ==============

try:
    # 新版 openai SDK（兼容 DeepSeek 的 base_url）
    from openai import OpenAI
except ImportError:
    raise ImportError(
        "请先安装 openai SDK:  pip install openai>=1.0.0\n"
    )

In [11]:
class LLMClient:

    def __init__(
        self,
        api_key: Optional[str] = None,
        model_name: str = "deepseek-chat",
        base_url: str = "https://api.deepseek.com",
        temperature: float = 0.3,
    ):
       
        self.api_key = api_key or os.getenv("DEEPSEEK_API_KEY")
        if not self.api_key:
            raise ValueError(
                "未找到 DeepSeek API Key，请在环境变量中设置 DEEPSEEK_API_KEY"
            )

        self.model_name = model_name
        self.temperature = temperature
        
        self.client = OpenAI(api_key=self.api_key, base_url=base_url)

    def chat(self, messages: List[Dict[str, str]], temperature: Optional[float] = None) -> str:
       
        resp = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            temperature=temperature if temperature is not None else self.temperature,
        )
        content = resp.choices[0].message.content
        return content.strip()


In [12]:
# ============== 二、Query 改写模块 ==============

class QueryRewriter:

    def __init__(self, llm_client: LLMClient):
        self.llm = llm_client

        # 可以根据项目需要进一步调整 system prompt
        self.system_prompt = (
            "你是一名资深的中文临床医生助理，负责将用户的自然语言健康咨询问题改写为"
            "精准、专业、便于检索和医生理解的中文问题。"
            "改写要求："
            "1）保留与疾病、症状、检查、用药相关的关键信息；"
            "2）去除闲聊、情绪化表达、与医疗无关的内容；"
            "3）尽量用医学上常见的表达方式，但不要加入原文中没有的硬事实；"
            "4）如果能判断，就补全必要的前提信息（如成人/儿童、是否有既往病史等），"
            "   但要用“可能”“需要评估”等谨慎措辞；"
            "5）只输出【改写后的问题】一句话，不要额外解释。"
        )

    def rewrite(self, user_query: str) -> str:
        
        user_prompt = (
            "原始问题：\n"
            f"{user_query}\n\n"
            "请根据上述原始问题，输出一条更专业、简洁、适合用来检索和医生阅读的【改写后的问题】。"
        )

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_prompt},
        ]

        rewritten = self.llm.chat(messages)
       
        rewritten = rewritten.replace("【改写后的问题】", "").strip("：: \n")
        return rewritten


In [13]:
# ============== 三、关键词/科室/疾病 提取模块 ==============

class KeywordExtractor:
    
    def __init__(self, llm_client: LLMClient):
        self.llm = llm_client

        self.system_prompt = (
            "你是一名熟悉中国医院科室划分的临床医生助理，"
            "需要根据用户的中文描述，推断应该去哪个科室就诊，并抽取与疾病相关的关键词。\n"
            "请严格按照 JSON 格式输出，不能包含任何多余说明文字。"
            "JSON 字段定义如下：\n"
            "{\n"
            '  "department": "就诊科室名称（如：消化内科、心内科、神经内科、妇产科、儿科、皮肤科等）",\n'
            '  "diseases": ["可能相关的疾病或诊断名称，数组形式，可以为空数组"],\n'
            '  "keywords": ["用于检索的关键词或短语，数组形式，可以为空数组"]\n'
            "}\n"
            "注意：\n"
            "1）如果不能确定具体科室，可以给出最可能的一个，例如“内科门诊”；\n"
            "2）疾病名称尽量用临床常用的标准中文术语；\n"
            "3）keywords 中可以包含症状、检查名称、药物名称等；\n"
            "4）一定要输出严格 JSON，不要带注释、不带中文标点。"
        )

    def _parse_json_from_llm(self, text: str) -> Dict[str, Any]:
        
        text = text.strip()

        # 1. 先尝试直接解析
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            pass

        # 2. 用正则从文本中提取第一个 {...} 块再解析
        match = re.search(r"\{.*\}", text, re.S)
        if match:
            json_str = match.group(0)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError:
                pass

        # 3. 实在解析不了，返回一个兜底结构
        return {
            "department": "",
            "diseases": [],
            "keywords": [],
            "raw_text": text,  
        }

    def extract(self, user_query: str) -> Dict[str, Any]:
        
        user_prompt = (
            "用户原始问题：\n"
            f"{user_query}\n\n"
            "请根据上面的描述，推断就诊科室并抽取疾病与关键词，"
            "只输出一个 JSON 对象。"
        )

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_prompt},
        ]

        resp_text = self.llm.chat(messages)
        result = self._parse_json_from_llm(resp_text)

        result.setdefault("department", "")
        result.setdefault("diseases", [])
        result.setdefault("keywords", [])

        if not isinstance(result["diseases"], list):
            result["diseases"] = [str(result["diseases"])]
        if not isinstance(result["keywords"], list):
            result["keywords"] = [str(result["keywords"])]

        return result


In [14]:
# ============== 四、组合封装 ==============

class MedicalQueryProcessor:

    def __init__(self, llm_client: Optional[LLMClient] = None):
        # 允许传入已有的 llm_client，也可以在内部默认初始化一个
        self.llm = llm_client or LLMClient()
        self.rewriter = QueryRewriter(self.llm)
        self.extractor = KeywordExtractor(self.llm)

    def rewrite_query(self, user_query: str) -> str:
        return self.rewriter.rewrite(user_query)

    def extract_keywords(self, user_query: str) -> Dict[str, Any]:
        return self.extractor.extract(user_query)

    def process(self, user_query: str) -> Dict[str, Any]:
       
        rewritten = self.rewriter.rewrite(user_query)
        kw_result = self.extractor.extract(user_query)

        return {
            "original_query": user_query,
            "rewritten_query": rewritten,
            "department": kw_result.get("department", ""),
            "diseases": kw_result.get("diseases", []),
            "keywords": kw_result.get("keywords", []),
        }


In [16]:
# ============== 五、简单示例：本文件直接运行时的 demo ==============

if __name__ == "__main__":
    
    user_query = (
        "我有高血压这两天女婿来的时候给我拿了些党参泡水喝，您好高血压可以吃党参吗？"
    )

    processor = MedicalQueryProcessor()
    result = processor.process(user_query)

    print("====== 原始问题 ======")
    print(result["original_query"])
    print("\n====== 改写后的问题 ======")
    print(result["rewritten_query"])
    print("\n====== 推断科室 ======")
    print(result["department"])
    print("\n====== 可能相关疾病 ======")
    print(result["diseases"])
    print("\n====== 检索关键词 ======")
    print(result["keywords"])

====== 原始问题 ======
我有高血压这两天女婿来的时候给我拿了些党参泡水喝，您好高血压可以吃党参吗？

====== 改写后的问题 ======
高血压患者能否服用党参泡水？

====== 推断科室 ======
心内科

====== 可能相关疾病 ======
['高血压']

====== 检索关键词 ======
['党参', '泡水喝', '高血压']
